# Introduction to Neural Networks 

In this exercise we will learn how to use neural networks to solve regression problems. We will use a neural network to predict the currents of the actuators and the change in the joint angle of our Openmanipulator robot arm. In the lecture we learned that we need: 
- data to train the neural network
- pre and post processing functions to prepare the data for the neural network and to interpret the output of the neural network
- a neural network architecture
- a loss function to calculate the error between the output of the neural network and the desired output
- a forward function to calculate the output of the neural network
- a Optimize function to update the weights of the neural network

To build and train the neural network we will use the **flux.jl** package. This package provides a lot of functions to build and train neural networks. The documentation of the package can be found [here](https://fluxml.ai/Flux.jl/stable/).

## Predicting currents in the Openmanipulator joints

In this part we will use a neural network to predict the currents in our Openmanipulator robot arm. We will use real measured data of the currents and the joint angles of the first joint of our robot to train a neural network. First we will import the data and plot it to get a better understanding of the data. Secondly we will preprocess the data and train a neural network. Finally we will evaluate the performance of our neural network.

In [ ]:
import Pkg                                          # Package manager
Pkg.generate(joinpath(@__DIR__, "NeuronaleNetze"))  # activate package environment
Pkg.activate(joinpath(@__DIR__, "NeuronaleNetze"))  # activate package environment
Pkg.add("CSV")                                      # add CSV
Pkg.add("Plots")                                    # add Plots
Pkg.add("Statistics")                               # add Statistics
Pkg.add("ReverseDiff")                              # add ReverseDiff
Pkg.add("Flux")                                     # add Machine Learning Library Flux

In [ ]:
import Pkg                                          # Package manager
Pkg.activate(joinpath(@__DIR__, "NeuronaleNetze"))  # activate package environment
using CSV                                           # CSV Parser
using Plots                                         # Graph Plotter
using Statistics                                    # statisitical functions, like "mean"
using Flux                                          # Machine Learning Library
using ReverseDiff                                   # Reverse Mode Automatic Differentiation

### The Data

Using the Openmanipulator we collected data using the Dynamixel Wizard tool which enables us to control and measure each actuator of our Robot. An image of the measured test data is shown below. 

<img src="./OMP_Daten/pc-test.png" width="800">

The data is stored in a CSV file. To load the CSV file, we can use the CSV module from Julia. For the input $x$ into our model we will read the columns for: 
- Present Position
- Goal Position
- Present Velocity

We will make build the input matrix $x$ of these values.

For the output $y$ we will read the columns for:
- Present Current

We will also make a matrix of these values. Since we are only predicting one value, we will only have one column. However, Flux expects a matrix, so we will make a matrix with one column.


In [ ]:
function load_csv_data(filename)
    csv = CSV.File(filename)                        # load and parse csv
    x_1 = csv.columns[3].column                     # mask column 3 (Present Position) - input 1
    x_2 = csv.columns[4].column                     # mask column 4 (Goal Position) - input 2
    x_3 = csv.columns[7].column                     # mask column 7 (Present Velocity) - input 3
    y = csv.columns[5].column                       # mask column 5 (Present Current) - this is what we want to predict
    x_train = hcat(x_1, x_2, x_3)'                  # create input matrix
    y_train = y                                     # create output vector
    y_train = reshape(y_train, 1, :)                # reshape output vector to a matrix (1 x n) - this is what Flux expects
    x_train, y_train
end

In [ ]:
# load training data
x_train, y_train = load_csv_data(joinpath(@__DIR__, "OMP_Daten/pc-train.csv"))

# load test data
x_test, y_test = load_csv_data(joinpath(@__DIR__, "OMP_Daten/pc-test.csv"))

To normalize the data we estimate the mean and standard deviation of the data. 

In [ ]:
mean_x = mean(x_train)
mean_y = mean(y_train)
std_x = std(x_train)
std_y = std(y_train)

### Task: Functions for preprocessing and postprocessing the data

1. Write a function to normalize the data using the mean and standard deviation.

2. Write a function to denormalize the data using the mean and standard deviation.

In [ ]:
# data pre procseeing (make it machine frindly)
function dataPreProcess(x, y)
    x = Float32.(x)
    y = Float32.(y)
    ...
    x, y
end

# data post procseeing (make it human frindly)
function dataPostProcess(x, y)
    ...
    x, y
end

Preprocess the data using the function you wrote.

In [ ]:
# Data Preprocessing
x_train, y_train = dataPreProcess(x_train, y_train)

Check the mean and std of the data to make sure we did it right. 

In [ ]:
@show mean(x_train)
@show std(x_train)
@show mean(y_train)
@show std(y_train);

We also need to normalize the test data. It is important to use the mean and std of the training data to normalize the test data since our neural network is trained on the normalized training data. Using a different mean and std for the test data would mean that we are using a different distribution for the test data than for the training data.

In [ ]:
x_test, y_test = dataPreProcess(x_test, y_test)

## Starting with a Linear Model 

In the lecture we showed that perceptrons can be build out of simple linear functions and adding a nonlinear activation layer. So before we will implement a neural network we take a step back and implement a linear function which estimates the currents. 

The linear function is defined as:
$$  
\tilde{y} = \tilde{x}^T \cdot \tilde{w} + b
$$
where $x$ is the input vector and $w$ is the weight vector and $b$ is the bias. We changed the notation to avoid adding the bias as a additional weight: 
$$
y = x^T \cdot w = \begin{bmatrix} 1 & x_1 & x_2 & \dots & x_n \end{bmatrix} \cdot \begin{bmatrix} w_b \\ w_1 \\ w_2 \\ \dots \\ w_n \end{bmatrix} = 1 \cdot w_b + x_1 \cdot w_1 + x_2 \cdot w_2 + \dots + x_n \cdot w_n
$$
where $w$ is now the extended weight vector and $x$ is the input vector extended by a 1.

First lets look at an example of how to calculate the output of the linear function. We will use the first 3 rows of the training data as an example. The weight vector $w$ is used to generate an output $y$ for each row of the matrix $X$:

In [ ]:
x_1 = x_train[:, 1:3] # first 3 training examples each having 3 values
@show x_1 = vcat(ones(1, size(x_1, 2)), x_1)
@show w_1 = [1, 1, 1, 1]
@show x_1' * w_1;

### Task: Build a linear model

1. Build a linear model using the definition we gave above.

In [ ]:
# forward propagation
function linear_forward(x, w)
    x = ...

    # Calculate the output of the neuron
    y_hat = ...
end

# Define a function that takes in the weights and the data, and returns the loss
function loss(w, x, y)
    y_hat = ...
    mse = sum((y_hat .- y) .^ 2) / size(y, 1)
    return mse
end

# Define a function that computes the gradients of the loss with respect to the weights
function grad(w, x, y)
    return ReverseDiff.gradient(w -> loss(w, x, y), w)
end

# initialize model using random normal weights
function train_linear_model(x, y, learning_rate, num_epochs)
    w = randn(4, 1)
    mse = loss(w, x_t, y_t)
    println("Start: MSE = $mse")
    for i = 1:num_epochs
        # Compute the gradients of the loss with respect to the weights
        dw = grad(w, x, y)

        # Update the weights using gradient descent
        w -= learning_rate * dw

        if i % 10 == 0
            # Compute the loss and print it
            mse = loss(w, x, y)
            println("Iteration $i: MSE = $mse")
        end
    end
    return w
end

Before we start to train a linear model on the real dataset we will generate an artificial one where we know that we can actually learn it using a linear function. We will use the following to generate the data:

In [ ]:
using Random

# Set the random seed for reproducibility
Random.seed!(1234)

# Define the true weights
w_true = randn(3, 1)

# Generate the dataset
n_samples = 1000
x_t = randn(3, n_samples)
y_t = x_t' * w_true

# Add some noise to the targets
y_t += 0.1 * randn(n_samples, 1);

w = train_linear_model(x_t, y_t, 0.01, 200)

@show w_true
@show w;

### Task: Train the linear model on the current dataset 
Now lots try to train the model on the real data. We already extracted the data from the CSV file and normalized it. So we can just start the training process by passing `x_train`and `y_train` to the `train_linear_model` function.

In [ ]:
train_linear_model(x_train, y_train, 0.01, 200)

Ok, this does not seem to work. Can you think of a reason why? 

### Task: Build a linear model using Flux
We can also build and train a linear model in a few lines of code using FLux. Build a linear model using Flux and train it on the data.

In [ ]:
using Flux
# building a linear  model using Flux
linear_model = ...
                    

# Define the loss function
loss(x, y) = sum((linear_model(x) .- y) .^ 2) / size(y, 1)

# Define the optimizer
opt = Descent(0.01) # you can also try it with the ADAM optimizer 

# Train the model - check the train function and what we pass to it
# Note that the train function is called with train! which means that it will modify the model parameters passed to it
numIter = 200
for i in 1:numIter
    Flux.train!(loss, Flux.params(linear_model), [(x_train, y_train)], opt, cb=() -> println(loss(x_train, y_train)))
end

### Linear model with nonlinear activation function using Flux
Now we extend the linear model by adding a nonlinear activation function. We will use the sigmoid function as activation function using `Flux.sigmoid`. Our function is then defined as:
$$
y = h(x^T \cdot w) = h(\begin{bmatrix} 1 & x_1 & x_2 & \dots & x_n \end{bmatrix} \cdot \begin{bmatrix} w_b \\ w_1 \\ w_2 \\ \dots \\ w_n \end{bmatrix}) = h(1 \cdot w_b + x_1 \cdot w_1 + x_2 \cdot w_2 + \dots + x_n \cdot w_n)
$$
where $h$ is the sigmoid function.

In [ ]:
# build linear model with nonlinear sigmoid activation function
non_linear_model = ...

# Define the loss function
loss(x, y) = sum((non_linear_model(x) .- y) .^ 2) / size(y, 1)

# Define the optimizer
opt = Descent(0.01) # you can also try it with the ADAM optimizer 

# Train the model
numIter = 200
for i in 1:numIter
    Flux.train!(loss, Flux.params(non_linear_model), [(x_train, y_train)], opt, cb=() -> println(loss(x_train, y_train)))
end

## Neural Network 

Now we have seen that we cannot learn the current of the actuators using a linear model. We also adapted the linear model with a nonlinear activation function. Now we will use several layers of these linear functions with nonlinear activation function to build a feed forward neural network. We will use a fully connected feed forward neural network with 1-2 hidden layers and a linear output layer. To build such a fully connected feed forward neural network we can use the `chain` function from Flux. The `chain` function takes a list of layers as input and builds a neural network with these layers. In this case we will use `Dense` layers which are fully connected layers. The `Dense` layer takes the number of input neurons, the number of output neurons and the activation function as input.

### Task: Build the neural network

Build a neural network with 1-2 hidden layer. The input layer should have 3 neurons and the output layer should have 1 neuron to fit our data. You can also test different activation functions and different numbers of neurons in the hidden layer. 

You can check the different activation function in flux here: https://fluxml.ai/Flux.jl/stable/models/activation/ 

To optimize the neural network we will use the ADAM optimizer and set its learning rate to $0.001$. You can check the different optimizers in flux here: https://fluxml.ai/Flux.jl/stable/training/optimisers/

In [ ]:
# define our Neural Network model
function build_model()
    opt = ...
    input_size = size(x_train)[1]
    output_size = size(y_train)[1]
    model = Chain(Dense(input_size, ...),
        ...,
        Dense(..., output_size, identity))
    model, opt
end

### Task: define the forward function

We also define a forward function to calculate the output of the neural network.

In [ ]:
function forward(x)
    y_hat = model(x)
end

### Task: define the loss function

We also need a loss function to optimize. Since we are doing regression, we will use the mean squared error loss function. You can check the different loss functions in flux here: https://fluxml.ai/Flux.jl/stable/models/losses/

In [ ]:
# Mean square error as the loss function for optimization
function loss(y_hat, y)
    sum((y_hat .- y) .^ 2)
end

### Task: Train the neural network

Now we will put everything together and train the neural network. We will define a function which takes the neural network, the optimizer, and the number of epochs as input. The function should return the loss based on the training and test data.

In [ ]:
# train the model
function train_model(model, opt, numIter)
    trainLoss = zeros(numIter)
    testLoss = zeros(numIter)
    opt_state = Flux.setup(opt, model)
    for i in 1:numIter
        Flux.train!(model, [(x_train, y_train)], opt_state) do m, x, y
            loss(m(x), y)
        end
        trainLoss[i] = loss(forward(x_train), y_train) / length(y_train)
        testLoss[i] = loss(forward(x_test), y_test) / length(y_test)
    end
    trainLoss, testLoss
end

Calling the `build_model()` and `train_model()` functions:

In [ ]:
model, opt = build_model() # build model
trainLoss, testLoss = train_model(model, opt, 1000) # train model

Plotting the train and test loss:

In [ ]:
# plot the train loss 
plot(trainLoss, label="Train Loss", xlabel="Iterations", ylabel="loss")

# plot the test loss
plot!(testLoss, label="Test Loss", xlabel="Iterations", ylabel="loss")

Let's also plot the predicted currents and the actual currents based on the training data:

In [ ]:
# plot the results for the training data
y_hat = forward(x_train)
plot(y_train[1, :], label="Ground Truth (train)", ylabel="Current")
plot!(y_hat[1, :], label="Prediction (train)")

Let's do the same for the test data:

In [ ]:
# test the final model on the test data
y_hat = forward(x_test)
error = sum((y_hat .- y_test) .^ 2) / length(y_test)
println("Test Error: ", error)

# post process the data (make it human readable)
x_test, y_test = dataPostProcess(x_test, y_test)
_, y_hat = dataPostProcess(x_test, y_hat)

# plot the results
plot(y_test[1, :], label="Ground Truth (test)", ylabel="Current")
plot!(y_hat[1, :], label="Prediction (test)")

# error in human readable form
error = y_hat .- y_test

# plot the error
plot!(error[1, :], label="Error", ylabel="Error")

## Could we replace a simulation with a neural network?

In this part we will investigate how well a neural network can predict the change of the joint angle of the first joint of the Openmanipulator-X based on the joint angle, the goal joint angle and the current in the actuator. 

In [ ]:
function load_csv_data(filename)
    csv = CSV.File(filename)                        # load and parse csv
    x_1 = csv.columns[3].column[1:end-1]            # mask column 3 (Present Position) 
    x_2 = csv.columns[4].column[1:end-1]            # mask column 4 (Goal Position) 
    x_3 = csv.columns[5].column[1:end-1]            # mask column 5 (Present Current) 
    # the prediction will be the delta position of the joint (Present Position at time t+1 - Present Position at time t)
    y = csv.columns[3].column[2:end] - csv.columns[3].column[1:end-1] # mask column 3 (delta Position) - this is what we want to predict

    x_train = hcat(x_1, x_2, x_3)'                  # create input matrix
    y_train = y                                     # create output vector
    y_train = reshape(y_train, 1, :)                # reshape output vector to a matrix (1 x n) - this is what Flux expects
    x_train, y_train
end

In [ ]:
# load training data
x_train, y_train = load_csv_data(joinpath(@__DIR__, "OMP_Daten/pc-train.csv"))
# load test data
x_test, y_test = load_csv_data(joinpath(@__DIR__, "OMP_Daten/pc-test.csv"))

Lets plot the training data to get a better understanding of the data:

In [ ]:
# plot the data and set the heading 
p1 = plot(x_train[1, :], label="P-Pos", ylabel="x_1")
p2 = plot(x_train[2, :], label="G-Pos", ylabel="x_2")
p3 = plot(x_train[3, :], label="P-I", ylabel="x_3")
p4 = plot(y_train[1, :], label="∂-Pos", ylabel="y")

plot(p1, p2, p3, p4, layout=(4, 1))

Normalize the data: 

In [ ]:
# calculate mean and standard deviation of the training data
mean_x = mean(x_train)
mean_y = mean(y_train)
std_x = std(x_train)
std_y = std(y_train)
# Data Preprocessing
x_train, y_train = dataPreProcess(x_train, y_train)
x_test, y_test = dataPreProcess(x_test, y_test)

Now we build the model and train it. This time we will train it for $20000$ epochs. The rest of the parameters stay the same.

In [ ]:
model, opt = build_model() # build model
trainLoss, testLoss = train_model(model, opt, 20000) # train model

Let's plot the train and test loss:

In [ ]:
# plot the train loss
fig = plot(1:numIter, trainLoss, label="Train Loss", xlabel="Iterations", ylabel="loss")
# plot the test loss
plot!(1:numIter, testLoss, label="Test Loss", xlabel="Iterations", ylabel="loss")

Check how well the model predicts the joint angle change based on the training data:

In [ ]:
# plot the results for the training data
y_hat = forward(x_train)
plot(y_train[1, :], label="Ground Truth (train)", ylabel="Position")
plot!(y_hat[1, :], label="Prediction (train)")

Let's do the same for the test data:

In [ ]:
# test the final model on the test data
y_hat = forward(x_test)
error = sum((y_hat .- y_test) .^ 2) / length(y_test)
println("Test Error: ", error)

# post process the data (make it human readable)
x_test, y_test = dataPostProcess(x_test, y_test)
_, y_hat = dataPostProcess(x_test, y_hat)

# plot the results
plot(y_test[1, :], label="Ground Truth (test)", ylabel="Position")
plot!(y_hat[1, :], label="Prediction (test)")

# error in human readable form
error = y_hat .- y_test

# plot the error
plot!(error[1, :], label="Error")

In [ ]:
# test the final model on the test data
y_hat = forward(x_test)
error = sum((y_hat .- y_test) .^ 2) / length(y_test)
println("Test Error: ", error)

# post process the data (make it human readable)
x_test, y_test = dataPostProcess(x_test, y_test)
_, y_hat = dataPostProcess(x_test, y_hat)

# plot the results
plot(y_test[1, :], label="Ground Truth (test)", ylabel="Position")
plot!(y_hat[1, :], label="Prediction (test)")

# error in human readable form
error = y_hat .- y_test

# plot the error
plot!(error[1, :], label="Error")